In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["KAGGLE_USERNAME"] = 'hyunw3'
os.environ["KAGGLE_KEY"] = "b48a329f42b53a6bda2afd4cf341bcf2"
# https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-quickstart.html
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
torch.cuda.set_per_process_memory_fraction(0.3,0)

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
train_data = TabularDataset("./train.csv")
subsample_size = 500
train_data.sample(n=subsample_size, random_state=0)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
157105,157105,0.225628,0.369954,0.086533,0.220996,0.413357,0.406488,0.502186,0.535051,0.398884,...,1,0,0,0,0,0,1,0,0,1
374554,374554,0.156088,0.479218,0.087969,0.335879,0.601985,0.492081,0.361206,0.726592,0.691247,...,0,1,1,0,0,0,0,0,0,0
688694,688694,0.246660,0.465889,0.022856,0.310019,0.562853,0.512969,0.393365,0.600628,0.523308,...,0,0,0,0,0,0,0,0,0,0
265381,265381,0.253541,0.495103,0.236774,0.215441,0.540664,0.422883,0.409836,0.808747,0.353328,...,0,0,0,1,1,0,0,0,0,0
955415,955415,0.289054,0.378237,0.006109,0.272483,0.748737,0.515908,0.362741,0.629370,0.444692,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26972,26972,0.245825,0.396460,0.245351,0.232621,0.776057,0.334820,0.817839,0.652080,0.622675,...,0,0,0,0,1,0,0,0,0,1
460196,460196,0.161303,0.454262,0.014427,0.260020,0.589798,0.373510,0.475360,0.613104,0.344057,...,0,0,0,0,0,1,0,1,1,0
907019,907019,0.229526,0.571115,0.173815,0.508466,0.489185,0.386501,0.465174,0.630219,0.391914,...,0,0,0,0,0,0,0,0,0,0
229397,229397,0.181321,0.389418,0.096659,0.230548,0.606180,0.460271,0.389354,0.731110,0.617478,...,0,0,0,0,0,0,0,0,1,0


In [5]:
test_data = TabularDataset("./test.csv")
subsample_size = 500
test_data.sample(n=subsample_size, random_state=0)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
194257,1194257,0.177652,0.402826,0.170229,0.279291,0.449732,0.349957,0.450216,0.676548,0.431982,...,1,1,0,0,0,0,1,0,1,0
396319,1396319,0.326258,0.421919,0.162970,0.233764,0.669121,0.363471,0.422830,0.663648,0.598159,...,0,0,0,1,0,0,1,0,0,0
138738,1138738,0.207297,0.469348,0.164248,0.335177,0.483002,0.443952,0.358411,0.676115,0.550601,...,0,0,0,0,0,0,0,0,0,0
66570,1066570,0.203267,0.562071,0.367516,0.219651,0.656695,0.409316,0.476351,0.657574,0.371572,...,0,0,0,0,0,1,1,0,1,0
149424,1149424,0.227488,0.604065,0.090917,0.320712,0.577093,0.367012,0.455212,0.737396,0.531693,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176678,1176678,0.163133,0.388365,0.007658,0.348349,0.629932,0.330511,0.174350,0.609330,0.555974,...,0,0,0,0,0,0,0,0,0,0
448334,1448334,0.161898,0.353998,0.017979,0.245647,0.507021,0.454971,0.506786,0.709880,0.635846,...,0,0,1,0,1,0,0,1,0,0
458895,1458895,0.230312,0.390605,0.098201,0.210367,0.582872,0.387836,0.480212,0.589985,0.335449,...,1,0,0,1,0,0,0,1,0,0
351025,1351025,0.203836,0.616144,0.019308,0.272592,0.609978,0.433441,0.245892,0.662670,0.299882,...,1,0,0,0,1,0,0,0,0,0


In [6]:
label='target'
print("summary of class variable : \n",train_data[label].describe())

summary of class variable : 
 count    1000000.000000
mean           0.500485
std            0.500000
min            0.000000
25%            0.000000
50%            1.000000
75%            1.000000
max            1.000000
Name: target, dtype: float64


In [7]:
model_savepath = "./models_bagging/"
#if os.path.exists(model_savepath) is False:
#    os.mkdir(model_savepath)

In [ ]:
predictor = TabularPredictor(label=label, eval_metric='accuracy', path=model_savepath).fit(train_data,
                                        num_bag_folds=5, num_bag_sets=5, num_stack_levels=1
                                                                                          )

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "./models_bagging/"
AutoGluon Version:  0.3.2b20211005
Train Data Rows:    1000000
Train Data Columns: 286
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    484264.79 MB
	Train Data (Original)  Memory Usage: 2288.0 MB (0.5% of available memory)

In [ ]:
y_pred = predictor.predict(test_data)
y_pred

In [ ]:
test_id = test_data['id']
test_pred = pd.DataFrame(y_pred)
test_pred#.head()

In [ ]:
d_frame = pd.DataFrame()
d_frame["id"] = test_id
d_frame["target"] = y_pred
d_frame.to_csv("./submit_1005.csv",index=False)

In [ ]:
d_frame.head()

In [ ]:
predictor.leaderboard(train_data, silent=True) # max 0.76

In [ ]:
results = predictor.fit_summary(show_plot=True)
results # 0.7679